# Trabalho 2 de Finanças Quantitativas - UFMG

## Prof. Cristiano Arbex
## Eduardo Santiago Ramos

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

import csv
import urllib
import io
import requests
import pickle

import datetime
import scipy.stats as scs
import statsmodels.stats.stattools as smstools

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
py.init_notebook_mode(connected=True)

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 0. Manipulacao de dados

Fonte: Arquivo *dados.pdf*

## 0.1. Funcoes auxiliares

In [2]:
def plotscatter(df, columns=None, name='Stock name', title='Stock price', yaxis='Preco'):
    '''Imprime series temporais'''
    if columns is None: columns=df.columns
        
    # 1) Traces
    df_data = list(go.Scatter(x=df.index, y=df[col], name=col) for col in columns)

    # 2) Layout
    df_layout = go.Layout(title=title, legend={'orientation':'h'}, yaxis={'title':yaxis})

    # 3) Figure
    df_fig = go.Figure(data=df_data, layout=df_layout)
    py.iplot(df_fig)

In [3]:
def dados_do_trabalho(url, file_name):
    '''Le/baixa dados do trabalho 1 de Financas Quantitativas'''
    save_path = '{}.pkl'.format(file_name)
    try:
        with open(save_path, 'rb') as f:
            df = pickle.load(f)
            print(f'Dados lidos de {save_path}')
    except (OSError, IOError) as e:
        print('Baixando dados de {}'.format(url))
        s = requests.get(url).content
        df = pd.read_csv(io.StringIO(s.decode('utf-8')))
        df.to_pickle(save_path)
        print(f'Salvo em {save_path}')
        
    return df

## 0.2. Download dos dados

In [4]:
df_raw_daily = dados_do_trabalho("http://www.dcc.ufmg.br/~arbex/portfolios/IBOV.csv", 'ibov_diario')

Dados lidos de ibov_diario.pkl


In [5]:
df_raw_daily

,Date,IBOV,BOVA11,ABEV3,BVMF3,BBDC3,BBAS3,SANB11,BBSE3,BRML3,...,SAPR11,SMLS3,SUZB3,VIVT4,TIMP3,TAEE11,UGPA3,USIM5,VALE3,VVAR11
0,20070102,45382.61,0.00,2.68,0.00,9.82,11.05,0.00,0.00,0.00,...,0.00,0.00,0.00,22.07,9.84,2.59,15.16,21.66,0.00,0.00
1,20070103,44445.29,0.00,2.66,0.00,9.71,10.85,0.00,0.00,0.00,...,0.00,0.00,0.00,22.20,9.46,2.68,14.63,20.60,0.00,0.00
2,20070104,44019.77,0.00,2.66,0.00,9.75,11.25,0.00,0.00,0.00,...,0.00,0.00,0.00,22.32,9.04,2.65,14.45,20.27,0.00,0.00
3,20070105,42245.16,0.00,2.61,0.00,9.35,10.76,0.00,0.00,0.00,...,0.00,0.00,0.00,21.34,8.68,2.59,13.71,19.41,0.00,0.00
4,20070108,42829.93,0.00,2.66,0.00,9.65,11.01,0.00,0.00,0.00,...,0.00,0.00,0.00,21.75,8.62,2.62,13.84,20.14,0.00,0.00
5,20070109,42006.78,0.00,2.64,0.00,9.40,10.89,0.00,0.00,0.00,...,0.00,0.00,0.00,21.07,8.44,2.59,13.64,19.62,0.00,0.00
6,20070110,42335.67,0.00,2.63,0.00,9.36,10.86,0.00,0.00,0.00,...,0.00,0.00,0.00,21.16,9.16,2.59,14.07,20.37,0.00,0.00
7,20070111,42670.32,0.00,2.66,0.00,9.48,11.02,0.00,0.00,0.00,...,0.00,0.00,0.00,21.02,8.96,2.64,14.08,20.49,0.00,0.00
8,20070112,43094.97,0.00,2.72,0.00,9.66,11.06,0.00,0.00,0.00,...,0.00,0.00,0.00,21.35,9.08,2.59,14.11,21.06,0.00,4.11
9,20070115,42919.17,0.00,2.74,0.00,9.71,11.23,0.00,0.00,0.00,...,0.00,0.00,0.00,21.47,8.76,2.59,14.01,21.06,0.00,5.29


## 0.3. Transformacao de indices para datas

In [6]:
df_diario = df_raw_daily.copy() # faz copia do original

In [7]:
# Funcao que transforma data em formato inteiro em tipo datetime
dayint2date = lambda date_int: pd.to_datetime(date_int, format='%Y%m%d')

# Aplica na coluna 'Date'
df_diario['Date'] = df_diario['Date'].apply(dayint2date)

# Seta os indices para ser a coluna 'Date'
df_diario = df_diario.set_index('Date')

In [8]:
# Primeiras/ultimas linhas de cada data frame
df_diario

,IBOV,BOVA11,ABEV3,BVMF3,BBDC3,BBAS3,SANB11,BBSE3,BRML3,BBDC4,...,SAPR11,SMLS3,SUZB3,VIVT4,TIMP3,TAEE11,UGPA3,USIM5,VALE3,VVAR11
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-02,45382.61,0.00,2.68,0.00,9.82,11.05,0.00,0.00,0.00,10.34,...,0.00,0.00,0.00,22.07,9.84,2.59,15.16,21.66,0.00,0.00
2007-01-03,44445.29,0.00,2.66,0.00,9.71,10.85,0.00,0.00,0.00,10.27,...,0.00,0.00,0.00,22.20,9.46,2.68,14.63,20.60,0.00,0.00
2007-01-04,44019.77,0.00,2.66,0.00,9.75,11.25,0.00,0.00,0.00,10.29,...,0.00,0.00,0.00,22.32,9.04,2.65,14.45,20.27,0.00,0.00
2007-01-05,42245.16,0.00,2.61,0.00,9.35,10.76,0.00,0.00,0.00,9.86,...,0.00,0.00,0.00,21.34,8.68,2.59,13.71,19.41,0.00,0.00
2007-01-08,42829.93,0.00,2.66,0.00,9.65,11.01,0.00,0.00,0.00,10.22,...,0.00,0.00,0.00,21.75,8.62,2.62,13.84,20.14,0.00,0.00
2007-01-09,42006.78,0.00,2.64,0.00,9.40,10.89,0.00,0.00,0.00,10.05,...,0.00,0.00,0.00,21.07,8.44,2.59,13.64,19.62,0.00,0.00
2007-01-10,42335.67,0.00,2.63,0.00,9.36,10.86,0.00,0.00,0.00,9.90,...,0.00,0.00,0.00,21.16,9.16,2.59,14.07,20.37,0.00,0.00
2007-01-11,42670.32,0.00,2.66,0.00,9.48,11.02,0.00,0.00,0.00,10.05,...,0.00,0.00,0.00,21.02,8.96,2.64,14.08,20.49,0.00,0.00
2007-01-12,43094.97,0.00,2.72,0.00,9.66,11.06,0.00,0.00,0.00,10.30,...,0.00,0.00,0.00,21.35,9.08,2.59,14.11,21.06,0.00,4.11


## 0.4. Presenca de zeros

In [9]:
df_diario = df_diario.replace(0, np.nan) # troca 0 por NaN

# <span style='color:darkmagenta'>Questao 1.</span> 

Neste exercício vamos selecionar portfolios baseados em Markowitz e otimização de CVaR. A
ideia é escolher portfolios compostos pelos ativos do índice iBOV no primeiro dia útil de Fevereiro 2018 e
ver como se comportariam no mês. Vamos utilizar os dois anos anteriores como dados de entrada, entre
os inícios de Fevereiro de 2016 e Fevereiro de 2018. Se quiserem, podem utilizar o seguinte código em R
para preparar os dados:

In [10]:
prices = df_diario.copy()

# Inicio e fim das series in-sample e out-of-sample
inSampleInicio = prices.index[prices.index>='2016-02'][0]
inSampleFim = prices.index[prices.index<'2018-02'][-1]
outOfSampleInicio = prices.index[prices.index>='2018-02'][0]
outOfSampleFim = prices.index[-1]

# Eliminando empresas que possuem precos invalidos
iISI = prices.index.get_loc(inSampleInicio)
iISF = prices.index.get_loc(inSampleFim)
prices = prices.dropna(how='any', axis=1, subset=prices.index[iISI:iISF])

# Dados in-sample apenas das acoes do iBov
pricesIS = prices[inSampleInicio:inSampleFim].drop(columns=['IBOV','BOVA11'])
returnsIS = (pricesIS.diff() / pricesIS.shift(axis=0, periods=1)).iloc[1:]

# Dados out-of-sample apenas das acoes do iBov
pricesOS = prices[outOfSampleInicio:outOfSampleFim].drop(columns=['IBOV','BOVA11'])

# Outros dados out-of-sample
indexPriceOS = prices.loc[outOfSampleInicio:outOfSampleFim, 'IBOV']

# Numero de ativos e de cenarios
S, N = returnsIS.shape

pricesIS
# returnsIS
# pricesOS
# indexPriceOS

,ABEV3,BVMF3,BBDC3,BBAS3,SANB11,BBSE3,BRML3,BBDC4,BRAP4,BRKM5,...,RADL3,RAIL3,SMLS3,VIVT4,TIMP3,TAEE11,UGPA3,USIM5,VALE3,VVAR11
Date,,,,,,,,,,,,,,,,,,,,,
2016-02-01,17.33,9.94,15.10,13.02,11.95,20.21,7.99,14.10,3.17,20.59,...,41.56,1.99,30.03,31.08,6.18,14.30,0.94,8.94,3.06,14.38
2016-02-02,16.94,9.69,14.48,12.05,11.25,19.35,7.72,13.54,2.90,20.03,...,41.47,1.85,29.98,30.29,5.85,13.73,0.92,8.09,3.40,13.78
2016-02-03,17.05,9.86,15.40,12.21,11.47,19.84,7.82,14.19,3.04,20.73,...,42.60,1.80,30.28,30.93,5.87,13.67,0.94,8.48,3.68,14.05
2016-02-04,17.29,9.96,16.43,12.69,12.05,20.16,8.12,14.96,3.47,20.98,...,42.84,1.89,30.29,32.07,6.13,14.28,1.00,9.73,3.88,13.99
2016-02-05,17.09,9.79,15.92,12.59,12.21,20.64,8.22,14.81,3.47,20.92,...,42.02,2.30,30.48,32.16,5.82,14.40,0.99,9.63,3.91,13.92
2016-02-10,16.90,9.81,16.02,12.59,12.26,20.64,8.18,14.98,3.36,20.39,...,43.09,2.17,30.38,31.30,5.77,14.36,0.95,9.60,3.78,13.92
2016-02-11,16.81,9.69,15.75,12.01,11.92,19.87,8.18,14.59,3.19,20.24,...,42.10,2.06,28.97,30.45,5.46,14.27,0.84,9.29,3.63,13.60
2016-02-12,16.88,9.64,15.80,11.96,12.08,19.87,8.18,14.63,3.35,20.08,...,41.81,2.14,30.93,31.49,5.77,14.32,0.84,9.65,3.73,13.68
2016-02-15,16.88,9.86,15.87,12.09,12.25,20.22,8.30,14.63,3.47,20.12,...,41.86,2.21,29.99,31.40,5.86,14.20,0.88,9.89,4.13,13.37


### a. Utilizando como entrada os dados in-sample, plote um gráfico com a fronteira eficiente utilizando o modelo de Markowitz. Varie o retorno de 0 a 0.008 com intervalos 0.0001 e permita shorting. Uma dica é utilizar o pacote `quadprog` para resolver o modelo, para ajuda verifique o Exemplo 7.12 das Lecture notes.

### [Resp]

A formulação de programação quadrática para o problema é a seguinte:

\begin{align}
    &\underset{w}{\text{minimize}} &&\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}w_iw_j\sigma_{ij} &&\equiv \quad &&\frac{1}{2} \mathbf{w^T}\mathbf{\Sigma}\mathbf{w} \\
    &\text{sujeito a} &&\sum_{i=1}^{N}w_i\mu_i \geq \overline{\mu} &&\equiv &&\mathbf{\mu^T}\mathbf{w} \geq \overline{\mu}\\
    & &&\sum_{i=1}^{N}w_i = 1 &&\equiv &&\mathbf{1^T}\mathbf{w} = 1 \\
\end{align}

- A variável de decisão é $\mathbf{w}$. 
- Os parâmetros fixos do modelo - retornos médios $\mu_i$ e covariâncias $\sigma_{ij}$ - são obtidos do conjunto `returnsIS`.
- Já os parâmetros variável - retorno mínimo $\overline{\mu}$ - é definido no intervalo entre $0$ e $0.008$, com passos de $0.0001$, a fim de obter um gráficom com a fronteira eficiente.
- A fim de permitir *shorting*, $\mathbf{w}$ é irrestrita.

In [11]:
import quadprog

# Matriz de retornos para extrair parametros
RetMat = returnsIS.values.T

# Obtendo parametros fixos do modelo
Sigma = np.cov(RetMat)       # matriz de covariancia
mu = np.mean(RetMat, axis=1) # retornos medios por ativo

# Definindo retornos minimos
ret = np.arange(0,0.0081,0.0001)
risk = np.empty(ret.shape)
for i, mu_min in enumerate(ret):
    print('Solucionando para retorno minimo de %.4f' % mu_min)
    
    # Matrizes de otimizacao
    G = Sigma                           # matriz de custo quadratico
    a = np.zeros(N)                     # vetor de custo (nao existe)
    C = np.vstack([np.ones(N), mu]).T   # matriz de m restricoes
    b = np.array([1, mu_min])           # termos a direita
    meq = 1                             # primeira restricao e' de igualdade

    # Verificacao de dimensoes
    n = N
    m = 2
    assert G.shape==(n,n) and a.shape==(n,) and C.shape==(n,m) and b.shape==(m,)
    
    # Soluciona problema quadratico
    quad_sol = quadprog.solve_qp(G, a, C, b, meq)
    (w_opt,f_opt) = quad_sol[0:2]
    risk[i] = f_opt
    
    # Validacao e impressao de dados:
    print('Porcentagem do orcamento investido: %.2f %%' % (np.matmul(np.ones(N),w_opt)*100))
    print('Retorno do portfolio: %.5f ' % np.matmul(mu,w_opt))
    print('Variancia do portfolio: %.5f \n' % f_opt)



Solucionando para retorno minimo de 0.0000
Porcentagem do orcamento investido: 100.00 %
Retorno do portfolio: 0.00066 
Variancia do portfolio: 0.00002 

Solucionando para retorno minimo de 0.0001
Porcentagem do orcamento investido: 100.00 %
Retorno do portfolio: 0.00066 
Variancia do portfolio: 0.00002 

Solucionando para retorno minimo de 0.0002
Porcentagem do orcamento investido: 100.00 %
Retorno do portfolio: 0.00066 
Variancia do portfolio: 0.00002 

Solucionando para retorno minimo de 0.0003
Porcentagem do orcamento investido: 100.00 %
Retorno do portfolio: 0.00066 
Variancia do portfolio: 0.00002 

Solucionando para retorno minimo de 0.0004
Porcentagem do orcamento investido: 100.00 %
Retorno do portfolio: 0.00066 
Variancia do portfolio: 0.00002 

Solucionando para retorno minimo de 0.0005
Porcentagem do orcamento investido: 100.00 %
Retorno do portfolio: 0.00066 
Variancia do portfolio: 0.00002 

Solucionando para retorno minimo de 0.0006
Porcentagem do orcamento investido: 100

#### Exemplo de alocacao: retorno minimo = 0.008

In [12]:
trace1 = go.Bar(
    y=returnsIS.columns,
    x=w_opt,
    name='Alocacao',
    orientation = 'h',
    marker = dict(
        color = 'rgba(246, 78, 139, 0.6)',
        line = dict(
            color = 'rgba(246, 78, 139, 1.0)',
            width = 3)
    )
)

data = [trace1]
layout = go.Layout(
    barmode='stack',
    title='Alocacao para retorno = 0.008'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')

In [13]:
# 1) Traces
front_trace = go.Scatter(x=np.sqrt(risk), y=ret, name='Fronteira de Portfolios')

# 2) Layout
front_layout = go.Layout(title='Fronteira de Portfolios', yaxis={'title':'Retorno'}, xaxis={'title':'Risco'})

# 3) Figure
df_fig = go.Figure(data=[front_trace], layout=front_layout)
py.iplot(df_fig)

### b. Resolva o modelo de Markowitz sem shorting e com retorno mínimo 0. Simule um portfolio iniciando em RS10000 com as proporções de Markowitz desde o primeiro até o último dia out-of-sample. Você pode utilizar a matriz `pricesOS` para fazer esta simulação.

### [Resp]

Para remover a possibilidade de *shorting*, o modelo do item (a) é modificado para requerer $w_i \geq 0$.

In [14]:
import quadprog

# Matriz de retornos para extrair parametros
RetMat = returnsIS.values.T

# Obtendo parametros fixos do modelo
Sigma = np.cov(RetMat)       # matriz de covariancia
mu = np.mean(RetMat, axis=1) # retornos medios por ativo

# Define retorno minimo
mu_min = 0

# Matrizes de otimizacao
G = Sigma                                           # matriz de custo quadratico
a = np.zeros(N)                                     # vetor de custo (nao existe)
C = np.vstack([np.ones(N), mu, np.identity(N)]).T   # matriz de m restricoes
b = np.r_[np.array([1, mu_min]), np.zeros(N)]       # termos a direita
meq = 1                                             # primeira restricao e' de igualdade

# Verificacao de dimensoes
n = N
m = N+2
assert G.shape==(n,n) and a.shape==(n,) and C.shape==(n,m) and b.shape==(m,)

# Soluciona problema quadratico
quad_sol = quadprog.solve_qp(G, a, C, b, meq)
(w_opt,f_opt) = quad_sol[0:2]

# Validacao e impressao de dados:
print('Porcentagem do orcamento investido: %.2f %%' % (np.matmul(np.ones(N),w_opt)*100))
print('Retorno do portfolio: %.5f ' % np.matmul(mu,w_opt))
print('Variancia do portfolio: %.5f \n' % f_opt)

w_mark = w_opt

Porcentagem do orcamento investido: 100.00 %
Retorno do portfolio: 0.00101 
Variancia do portfolio: 0.00003 



#### Alocacao do valor

In [15]:
trace1 = go.Bar(
    y=returnsIS.columns[w_mark>1e-4],
    x=w_mark[w_mark>1e-4],
    name='Alocacao',
    orientation = 'h',
    marker = dict(
        color = 'rgba(246, 78, 139, 0.6)',
        line = dict(
            color = 'rgba(246, 78, 139, 1.0)',
            width = 3)
    )
)

data = [trace1]
layout = go.Layout(
    barmode='stack',
    title='Alocacao sem shorting para retorno minimo = 0'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')

### Simulação do Portfolio

In [16]:
port_return = lambda w: np.matmul(w,(pricesOS.iloc[-1].values / pricesOS.iloc[0].values))

print('Retorno do portfolio com as proporcoes do Markowitz: %.2f %%' % ((port_return(w_mark)-1)*100))

Retorno do portfolio com as proporcoes do Markowitz: 2.19 %


### c. Resolva o modelo que otimiza o CVaR (Slide 60 em Downside Risk) com retorno mínimo zero, sem shorting e $\alpha = 10\%$. Faça a mesma simulação da letra b., mas desta vez com as proporções obtidas no modelo do CVaR.

### [Resp]

A formulação linear para o problema é a seguinte:

\begin{align}
    &\underset{w, \theta}{\text{minimize}} & &\theta + \frac{1}{S\alpha}\sum_{t=1}^{S}d_t &&\equiv 
    &&\theta + \frac{1}{S\alpha}\sum\mathbf{d} &&\equiv
    &&\left[1 \ |\ \mathbf{0}^{N^T}  \ |\ \mathbf{\frac{1}{S\alpha}}^{S^T} \right] \mathbf{x}\\
    %
    &\text{sujeito a} &&d_t \geq -\theta - \sum_{i=1}^{N}r_{it}w_i, \quad t \in S &&\equiv 
    &&-\theta - \mathbf{r_t}^T\mathbf{w}  -d_t\leq 0, \quad t \in S &&\equiv
    &&-\left[\mathbf{1}^{S} \ |\ \mathbf{R}  \ |\ \mathbf{I}_S\right] \mathbf{x} &&\leq \mathbf{0}^{S^T}\\
    %
    %& &&p_{t} = \sum_{i=1}^{N}r_{it}w_i, \quad t \in S &&\equiv 
    %&&- \mathbf{R}\mathbf{w} + \mathbf{p} = 0 &&\equiv
    %&&\left[\mathbf{0}^{S} \ |\ -\mathbf{R}  \ |\ \mathbf{0}_{SxS}  \ |\ \mathbf{I}_S \right] \mathbf{x} &&= \mathbf{0}^{S^T}\\
    %
    & &&\sum_{i=1}^{N}w_i\mu_i \geq \overline{\mu} &&\equiv &&\mathbf{\mu^T}\mathbf{w} \geq \overline{\mu} &&\equiv
    &&\left[0 \ |\ -\mathbf{\mu^T}  \ |\ \mathbf{0}^{S^T} \right] \mathbf{x} &&\leq - \overline{\mu}\\
    & &&\sum_{i=1}^{N}w_i = 1 &&\equiv &&\mathbf{1^T}\mathbf{w} = 1  &&\equiv
    &&\left[0 \ |\ \mathbf{1}^{N^T}  \ |\ \mathbf{0}^{S^T} \right] \mathbf{x} &&= 1\\
    %
    & &&w_i \geq 0  \\
    & &&d_t \geq 0  \\
    & &&\theta \geq 0  \\
    %
    &\text{tal que} && \mathbf{x} = \left[\theta \ |\ \mathbf{w}  \ |\ \mathbf{d} \right]\\
    & &&\mathbf{x} \in \mathbb{R}^{(1+N+S)}\\
    & &&\mathbf{R} \in \mathbb{R}^{SxN}
\end{align}

em que $\theta$ é o VaR no nível $\alpha$. 

- As variáveis de decisão são $w$ e $\theta$. 
- Os parâmetros fixos do modelo - $S$ cenários, retornos $r_{it}$ e retornos médios $\mu_i$ - são obtidos do conjunto `returnsIS`.
- Já os parâmetros variáveis do modelo - retorno mínimo $\overline{\mu}$ e nível $\alpha$ - são definido como $0$ e $0.10$, conforme o enunciado.

A formulação faz com que, quanto menor o nível $\alpha$, maior o peso dado à soma das diferenças $d_t$. A forma do modelo contrabalancear este efeito é aumentando o valor de $\theta$, o que, por sua vez, representa um maior Value at Risk.

In [17]:
import scipy.optimize as sco
# Minimize:     c^T * x

# Subject to:   A_ub * x <= b_ub
#               A_eq * x == b_eq

# Matriz de retornos para extrair parametros
R = returnsIS.values

# Obtendo parametros fixos do modelo
mu = np.mean(R, axis=0) # retornos medios por ativo

# Define retorno minimo e nivel do VaR
mu_min = 0
alpha = 0.1


# Vetor de decisao - x = [theta, w, d, p]
n = 1+N+S # numero de variaveis

# Funcao objetivo
c = np.r_[1, np.zeros(N), np.full(S,fill_value=(1/(S*alpha)))]

# Restricoes de desigualdade
A_ub_1 = -np.hstack([np.ones((S,1)), R, np.identity(S)])
A_ub_2 = np.hstack([0, -mu, np.zeros((S,))])
A_ub = np.vstack([A_ub_1, A_ub_2])
b_ub = np.r_[np.zeros(S,), -mu_min]

# Restricao de igualdade
A_eq = np.hstack([np.zeros((1,1)), np.ones((1,N)), np.zeros((1,S))])
b_eq = np.array([1])

# Bounds
bounds = [(0,None)] + [(0,None)]*N + [(0,None)]*S

# Verifica dimensoes
assert c.shape==(n,) and A_ub_1.shape==(S,n) and A_ub_2.shape==(n,) and A_ub.shape==(S+1,n) and \
       b_ub.shape==(S+1,) and A_eq.shape==(1,n) and b_eq.shape==(1,) and R.shape==(S,N) and mu.shape==(N,)
        
# Soluciona problema linear
sol = sco.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds,options={"disp": False})

# Extrai variaveis otimas
x_opt = sol.x
theta_opt = x_opt[0]  # VaR
w_opt = x_opt[1:1+N] # Alocacao (pesos w)

# Validacao e impressao de dados:
print('Porcentagem do orcamento investido: %.2f %%' % (np.matmul(np.ones(N),w_opt)*100))
print('Retorno do portfolio: %.5f ' % np.matmul(mu,w_opt))
print('VaR 10%%: %.5f \n' % theta_opt)

w_cvar = w_opt

Porcentagem do orcamento investido: 100.00 %
Retorno do portfolio: 0.00132 
VaR 10%: 0.00798 



#### Alocacao do valor

In [18]:
trace1 = go.Bar(
    y=returnsIS.columns[w_cvar>1e-4],
    x=w_cvar[w_cvar>1e-4],
    name='Alocacao',
    orientation = 'h',
    marker = dict(
        color = 'rgba(246, 78, 139, 0.6)',
        line = dict(
            color = 'rgba(246, 78, 139, 1.0)',
            width = 3)
    )
)

data = [trace1]
layout = go.Layout(
    barmode='stack',
    title='Alocacao sem shorting para retorno minimo = 0'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')

### Simulação do Portfolio

In [19]:
print('Retorno do portfolio com as proporcoes do CVaR: %.2f %%' % ((port_return(w_cvar)-1)*100))

Retorno do portfolio com as proporcoes do CVaR: 1.75 %


### d. Plote um gráfico contendo as performances normalizadas dos portfolios baseados em Markowitz e CVaR, além do índice iBov. O vetor `indexPricesOS` contém os dados do out-of-sample do índice.

In [20]:
n_mark = 10000*w_mark/(pricesOS.iloc[0].values) # quantidade de papeis de cada ativo - Markowitz
n_cvar = 10000*w_cvar/(pricesOS.iloc[0].values) # quantidade de papeis de cada ativo - CVaR
ibov_mult = 10000/indexPriceOS.iloc[0]

port_dict = {'portfolio_Mark':pricesOS.apply(lambda x: np.matmul(x,n_mark),axis=1),
             'portfolio_CVaR':pricesOS.apply(lambda x: np.matmul(x,n_cvar),axis=1),
             'valor_iBov':indexPriceOS.values*ibov_mult}
port_df = pd.DataFrame(data=port_dict, index=pricesOS.index)

In [21]:
plotscatter(port_df,yaxis='Valor do Portfolio',title='Desempenhos dos Portfolios Markowitz/CVaR e Indice iBov')

### e. Calcule, a partir dos valores out-of-sample obtidos, algumas medidas de performance dos portfolios e do índice:
1. **Variância**
- **CVaR 10%**
- **Sharpe ratio**
- **Sortino ratio**
- **Drawdown máximo**
- **Betas dos portfolios em relação ao índice.**

Observe que neste exercício não vamos rebalancear o portfolio periodicamente. Isto é, o portfolio é
comprado" no início e não mais alterado até o final. Calculamos o seu valor dia após dia até o final da
série.

### [Resp]

- **Variância**: $$\sum_{i=1}^{N}\sum_{j=1}^{N}w_iw_j\sigma_{ij}$$.
- **CVaR 10%**: $$CVaR_{\alpha} = \frac{1}{\alpha}\sum_{t=1}^{T}u_tR_t$$ em que os T cenários são obtidos de `pricesOS`. <br> <br>
- **Sharpe ratio**: $$Sh_P = \frac{\mu_P - R_f}{\sigma_P}$$ em que $\mu$ e $\sigma$ são obtidos das séries dos portfólios para o período *out of sample*. <br> <br>
- **Sortino ratio**: $$So_P = \frac{\mu_P - R_f}{\widehat{\sigma}_P}$$ em que $\mu$ e $\widehat{\sigma}$ são obtidos das séries dos portfólios para o período *out of sample*. <br> <br>
- **Drawdown máximo**: MDD = (Trough Value – Peak Value) ÷ Peak Value <br> <br>
- **Betas dos portfólios em relação ao índice**: $$\beta_P = \frac{\sigma_{ind,P}}{\sigma_{ind}^2}$$ em que o índice é o `IBOV`.

In [22]:
# Variancia
print('Variancias dos portfolios/iBov:')
print(port_df.var(),'\n\n')

# CVaR 10%
alpha = 0.1
OS = pricesOS.shape[0]
u_vec = np.full(int(np.floor(alpha*OS)),fill_value=1/OS) # pesos de cada um dos piores alpha cenarios
u_vec = np.r_[u_vec, alpha-np.sum(u_vec)]
returns_port = (port_df.diff() / port_df.shift(axis=0, periods=1)).iloc[1:]
cvar_ret = returns_port.apply(func=np.sort, axis=0).iloc[:u_vec.shape[0]]
cvar_df=pd.DataFrame(data=[np.matmul(u_vec,cvar_ret.values)],index=['CVaR_10%'],columns=returns_port.columns)
print('CVaR 10%% dos portfolios/iBov:')
print(cvar_df,'\n\n')

# Sharpe
Rf = 0
Sharpe = (returns_port.mean() - Rf)/(returns_port.std())
print('Sharpe Ratio dos portfolios/iBov:')
print(Sharpe,'\n\n')

# Sortino
Rf = 0
Sortino = (returns_port.mean() - Rf)/(returns_port.applymap(lambda x: min(x,0)).std())
print('Sortino Ratio dos portfolios/iBov:')
print(Sortino,'\n\n')

# Drawdown Maximo
def MDD(x):
    mdd=0
    for i in range(len(x)-1):
        for j in range(i+1,len(x)):
            if x[i]-x[j] > mdd: mdd=x[i]-x[j]
    return mdd
print('Maximum Drawdown dos portfolios/iBov:')
print(port_df.apply(func=MDD, axis=0),'\n\n')

# Betas em relacao ao indice
cov1 = np.cov(port_df['portfolio_Mark'].values, port_df['valor_iBov'].values)[0,1]
cov2 = np.cov(port_df['portfolio_CVaR'].values, port_df['valor_iBov'].values)[0,1]
print('Sortino Ratio dos portfolios/iBov (< 1 reduz volatilidade):')
print('Markowitz: %f' % (cov1 / np.var(port_df['valor_iBov'].values)))
print('CVaR: %f' % (cov2 / np.var(port_df['valor_iBov'].values)),'\n\n')

Variancias dos portfolios/iBov:
portfolio_CVaR    40019.174530
portfolio_Mark    40352.071883
valor_iBov        51567.289553
dtype: float64 


CVaR 10%% dos portfolios/iBov:
          portfolio_CVaR  portfolio_Mark  valor_iBov
CVaR_10%       -0.001428       -0.001414   -0.002227 


Sharpe Ratio dos portfolios/iBov:
portfolio_CVaR    0.106806
portfolio_Mark    0.132342
valor_iBov        0.002054
dtype: float64 


Sortino Ratio dos portfolios/iBov:
portfolio_CVaR    0.200802
portfolio_Mark    0.259830
valor_iBov        0.003684
dtype: float64 


Maximum Drawdown dos portfolios/iBov:
portfolio_CVaR    263.516029
portfolio_Mark    249.297975
valor_iBov        537.636949
dtype: float64 


Sortino Ratio dos portfolios/iBov (< 1 reduz volatilidade):
Markowitz: 0.865072
CVaR: 0.874062 




# <span style='color:darkmagenta'>Questao 2.</span> 

Considere um modelo de seleção de portfolios com shorting. Queremos limitar a quantidade
de shorting em no máximo 20% do valor total do portfolio. A solução dada pelo modelo (seja Markowitz
ou qualquer outro) deve garantir este limite. Como podemos fazer isso?

### [Resp]

#### Opção 1: 

Incluir variáveis de folga $l_i$ restritas da seguinte forma:
\begin{align}
&w_i + l_i &&\geq 0, \quad &&\forall i\\
&\sum_{i=1}^{N} l_i &&\leq 0.2 \\
&l_i &&\geq 0, \quad &&\forall i \\
\end{align}

#### Opção 2:

Desacoplar o shorting ($w_i \leq 0$) do long, fazendo a seguinte substituição:<br><br>
$$ w_i = w^{+}_i - w^{-}_i $$ <br>
tal que $w^{+}_i, w^{-}_i \geq 0$.

Com isto, basta incluir a restrição
$$ \sum_{i=1}^{N} w^{-}_i \leq 0.2$$

# <span style='color:darkmagenta'>Questao 3.</span> 

Por que, na prática, fundos que escolhem portfolios baseados em Markowitz também utilizam
o modelo de Black-Litterman?

### [Resp]

Pois ele ajuda a superar algumas dificuldades clássicas do modelo média-variância de Markowitz, tais como:
- portfolios altamente concentrados e não intuitivos;
- alta sensibilidade aos parâmetros de entrada;
- excesso de estimativas, com alta possibilidade de erros.

# <span style='color:darkmagenta'>Desafio.</span> 

Considere o modelo de otimização abaixo. O modelo assume, para cada ativo $i$, um portfolio atual composto por $X_i$ ações e proporções desejadas $w_i^*$, encontradas previamente, indicando como devemos dividir o novo portfolio. Considere também $P_i$ como o preço atual de $i$ e $f$ como o custo aplicado a cada transação e expresso como uma porcentagem do valor negociado.
O modelo utiliza variáveis $x_i$ indicando quantas ações de $i$ teremos após as negociações e $G_i$ indicando o valor financeiro gasto para alterarmos a composição de $i$ de $X_i$ para $x_i$ ações. Este modelo garante que gastaremos o mínimo possível nestes custos.


\begin{align}
    &\underset{\mathbf{x}}{\text{minimize}} & &\sum_{i=1}^{N}G_i\\
    %
    &\text{sujeito a} &&P_ix_i = w_i^* \left( \sum_{j=1}^{N}X_jP_j - \sum_{j=1}^{N}G_j \right) &&j = 1,..., N \\
    & &&G_i \geq (x_i - X_i)P_if &&j = 1,..., N \\
    & &&G_i \geq (X_i - x_i)P_if &&j = 1,..., N \\
\end{align}


O modelo inclui apenas custos variáveis (uma porcentagem do valor a ser negociado). Considere que,
além do custo variável, temos um custo fixo h, expresso em moeda, a ser aplicado a cada negociação.
Este valor é independente do tamanho da negociação. Altere o modelo acima para que inclua o custo
fixo.

### [Resp]

Se incluirmos uma variável binária que indique a existência de uma transação, podemos incluir o custo fixo:

\begin{align}
    &\underset{\mathbf{x}}{\text{minimize}} & &\sum_{i=1}^{N}G_i + h\sum_{i=1}^{N}H_i\\
    %
    &\text{sujeito a} &&P_ix_i = w_i^* \left( \sum_{j=1}^{N}X_jP_j - \sum_{j=1}^{N}G_j - h\sum_{j=1}^{N}H_j \right) &&j = 1,..., N \\
    & &&G_i \geq (x_i - X_i)P_if &&i = 1,..., N \\
    & &&G_i \geq (X_i - x_i)P_if &&i = 1,..., N \\
    & &&H_i \geq (X_i - x_i) / M &&i = 1,..., N \\
    & &&H_i \geq (X_i - x_i) / M &&i = 1,..., N \\
    & &&H_i \in \{0,1\}  &&i = 1,..., N \\
\end{align}

em que $M$ é um "número grande", que faça com que $|x_i - X_i|$ nunca seja maior que $1$. Um valor válido é então <br><br> $$M = \frac{\sum_{i=1}^{N}X_iP_i} / min(X_i)$$.